In [ ]:
# UPDATED: 28/11/2024
# https://plotly.com/python/choropleth-maps/
# https://plotly.com/python/tile-county-choropleth/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Clara del Rey/BIG DATA/Programacion/FTP/000 Datos/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Elecciones a la Comunidad de Madrid



Vamos a hacer un mapa coroplético con los datos de las elecciones a la Comunidad de Madrid por municipio.

Los datos de las elecciones se han obtenido de https://www.comunidad.madrid/dcma/decma/P/areadescarga_list.php.

Los datos con las formas de los municipios se han obtenido de https://www.madrid.org/nomecalles/DescargaBDTCorte.icm

# Librerías


Instalar geopandas y la última versión de Plotly (para featureidkey)


In [ ]:
!pip install geopandas

In [ ]:
!pip install  xlrd==2.0.1

# Formas de los municipios

Cargar los datos del shapefile

In [ ]:
# https://geopandas.org/en/latest/index.html

import geopandas as gpd
# path = '../datos/municipios.zip'

geodf = gpd.read_file(path +'Municipios.zip')
geodf.head()

,CODBDT,GEOCODIGO,DESBDT,geometry
0,981936,0014,La Acebeda,"POLYGON ((446133.839 4552287.918, 446207.88 45..."
1,981937,0029,Ajalvir,"POLYGON ((459091.64 4489528.638, 459091.358 44..."
2,981938,0035,Alameda del Valle,"POLYGON ((427742.39 4533545.712, 427742.806 45..."
3,981939,0040,El Álamo,"POLYGON ((419449.072 4453127.868, 419482.126 4..."
4,981940,0053,Alcalá de Henares,"POLYGON ((464488.659 4488254.2, 464489.724 448..."


Hay que convertir las coordenadas a lat/lon

In [ ]:
# El geocodigo sin el ultimo numero coincide con el nº de municipio
geodf.geometry = geodf.geometry.set_crs(epsg=3042)
geodf = geodf.to_crs(epsg=4326)
geodf.head()

,CODBDT,GEOCODIGO,DESBDT,geometry
0,981936,0014,La Acebeda,"POLYGON ((-3.64165 41.1201, -3.64077 41.12002,..."
1,981937,0029,Ajalvir,"POLYGON ((-3.48318 40.55551, -3.48319 40.55549..."
2,981938,0035,Alameda del Valle,"POLYGON ((-3.85852 40.94986, -3.85851 40.94984..."
3,981939,0040,El Álamo,"POLYGON ((-3.94677 40.22471, -3.94638 40.2246,..."
4,981940,0053,Alcalá de Henares,"POLYGON ((-3.41937 40.54428, -3.41936 40.54427..."


Lo guardamos como GeoJSON para luego leerlo con json.load()

In [ ]:
import json

geodf.to_file(path + "Municipios.gjson", driver = "GeoJSON")

with open(path + "Municipios.gjson", encoding='UTF-8') as geofile:
    j_file = json.load(geofile)

Vamos a hacer un mapa de datos aleatorios para ver las formas de los 179 municipios

In [ ]:
# Poligono: La Acebeda
acebeda = j_file['features'][0]
acebeda

{'type': 'Feature',
 'properties': {'CODBDT': 981936, 'GEOCODIGO': '0014', 'DESBDT': 'La Acebeda'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-3.641653963298946, 41.12010102440628],
    [-3.640771201903683, 41.12001909634126],
    [-3.639930490139107, 41.120096552840664],
    [-3.637948825191825, 41.1206274872101],
    [-3.636065797838684, 41.12076734009327],
    [-3.634920140371293, 41.120723120532105],
    [-3.633453842492047, 41.12089315387423],
    [-3.632428343416116, 41.12079564130756],
    [-3.631149439942435, 41.120233966087866],
    [-3.629878077081852, 41.118439560255425],
    [-3.628189147725263, 41.11761795253516],
    [-3.626501422619173, 41.11686882666915],
    [-3.624654115278535, 41.115980283369545],
    [-3.622995512373188, 41.114383640510184],
    [-3.621917202440418, 41.11358719349656],
    [-3.619062422817082, 41.11172326955275],
    [-3.617012708375266, 41.11016620050081],
    [-3.616060100917851, 41.10930840976998],
    [-3.615919303791165, 41.109233009

Vamos a hacer un mapa con datos aleatorios para ver las formas de los 175 municipios

Para asociar el dataframe y el JSON, hay que crear un campo común. El JSON no tiene id, asi que lo añadimos. Cogemos el campo GEOCODIGO sin el ultimo dígito porque nos viene bien para el siguiente comando

In [ ]:
for feature in j_file['features']:
  feature['id'] = int(feature['properties']['GEOCODIGO'][:-1])

In [ ]:
import pandas as pd
import numpy as np

ids = [x for x in range(1,180)]
ratio = np.random.rand(179)

datos = pd.DataFrame(ids)
datos.columns = ['ids']
datos['ratio'] = ratio

datos.head()

,ids,ratio
0,1,0.259857
1,2,0.195070
2,3,0.021952
3,4,0.327208
4,5,0.397950


In [ ]:
# Más estético y + rápido
datos = pd.DataFrame({
    'ids': range(1, 180),
    'ratio': np.random.rand(179)
})

datos.head()

,ids,ratio
0,1,0.116253
1,2,0.759353
2,3,0.457358
3,4,0.781523
4,5,0.760388


In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(datos,
                           geojson=j_file,
                           mapbox_style="carto-positron",
                           center = {"lat": 40, "lon": -3},
                           color = 'ratio',
                           height = 800,
                           zoom= 6,
                           opacity=0.5,
                           locations = "ids"
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

# Los datos de los resultados

In [ ]:
from numpy.random import random
import pandas as pd
import numpy as np

In [ ]:
excel2019 =  pd.ExcelFile(path + '2019_Mesas.xls')
datos2019 = pd.read_excel(excel2019, 'Municipios')
datos2019.head()

,Codcir,Codmun,Municipio,Unnamed: 3,Censo,Certif. Alta,Censo Total,Votos Totales,Votos Blancos,Votos Nulos,...,Cs,FE de las JONS,PCAS-TC,PUM+J,P-LIB,PP,PCTE,VOX,UPYD,PODEMOS-IU
0,28,1.0,La Acebeda,NaN,77,0,77,70,0,3,...,14,0,0,0,0,25,0,2,0,5
1,28,2.0,Ajalvir,NaN,3282,1,3283,2178,27,12,...,456,7,1,6,0,521,5,274,3,81
2,28,3.0,Alameda del Valle,NaN,169,0,169,140,0,0,...,19,0,0,0,0,41,0,8,0,3
3,28,4.0,El Álamo,NaN,6559,0,6559,4264,25,26,...,674,4,0,1,3,1102,2,537,5,236
4,28,5.0,Alcalá de Henares,NaN,136232,1,136233,89514,424,434,...,18120,132,39,88,28,15953,85,8094,138,5306


In [ ]:
datos2019.tail()

,Codcir,Codmun,Municipio,Unnamed: 3,Censo,Certif. Alta,Censo Total,Votos Totales,Votos Blancos,Votos Nulos,...,Cs,FE de las JONS,PCAS-TC,PUM+J,P-LIB,PP,PCTE,VOX,UPYD,PODEMOS-IU
176,28,901.0,Lozoyuela-Navas-Sieteiglesias,NaN,871,0,871,636,1,6,...,62,2,0,0,0,200,0,46,4,52
177,28,902.0,Puentes Viejas,NaN,522,0,522,402,1,4,...,43,0,0,0,0,123,0,47,0,44
178,28,903.0,Tres Cantos,NaN,35395,0,35395,27229,144,84,...,6316,16,7,26,13,5894,9,2135,44,1538
179,28,990.0,Residentes Ausentes Madrid,NaN,307406,0,307406,16366,59,34,...,3174,8,18,67,13,2808,24,1831,75,1273
180,Total,NaN,NaN,NaN,5059159,93,5059252,3251386,15020,13527,...,629940,2217,1794,3178,1246,719852,2610,287667,4057,181231


In [ ]:
# Antes hay que ocuparse de la última fila
datos2019 = datos2019[:-1].copy()

In [ ]:
# Vamos a cambiar Codmun a entero.
datos2019.Codmun = datos2019.Codmun.astype(np.int16)
datos2019.tail()

,Codcir,Codmun,Municipio,Unnamed: 3,Censo,Certif. Alta,Censo Total,Votos Totales,Votos Blancos,Votos Nulos,...,Cs,FE de las JONS,PCAS-TC,PUM+J,P-LIB,PP,PCTE,VOX,UPYD,PODEMOS-IU
175,28,183,Zarzalejo,NaN,1136,0,1136,821,3,13,...,77,1,1,0,2,193,0,60,0,109
176,28,901,Lozoyuela-Navas-Sieteiglesias,NaN,871,0,871,636,1,6,...,62,2,0,0,0,200,0,46,4,52
177,28,902,Puentes Viejas,NaN,522,0,522,402,1,4,...,43,0,0,0,0,123,0,47,0,44
178,28,903,Tres Cantos,NaN,35395,0,35395,27229,144,84,...,6316,16,7,26,13,5894,9,2135,44,1538
179,28,990,Residentes Ausentes Madrid,NaN,307406,0,307406,16366,59,34,...,3174,8,18,67,13,2808,24,1831,75,1273


In [ ]:
# Añadimos una columna con el año y otra con la participación en porcentaje
datos2019['convocatoria'] = 2019
print(datos2019.columns)

datos2019['participacion'] = 100 * datos2019['Votos Totales'] / datos2019['Censo Total']
datos2019.head()

Index(['Codcir', 'Codmun', 'Municipio', 'Unnamed: 3', 'Censo', 'Certif. Alta',
       'Censo Total', 'Votos Totales', 'Votos Blancos', 'Votos Nulos',
       'Abstención', 'Votos Válidos', 'Votos Candidaturas', 'PACMA',
       'MÁS MADRID', 'PSOE', 'PH', 'ULEG', 'Cs', 'FE de las JONS', 'PCAS-TC',
       'PUM+J', 'P-LIB', 'PP', 'PCTE', 'VOX', 'UPYD', 'PODEMOS-IU',
       'convocatoria'],
      dtype='object')


,Codcir,Codmun,Municipio,Unnamed: 3,Censo,Certif. Alta,Censo Total,Votos Totales,Votos Blancos,Votos Nulos,...,PCAS-TC,PUM+J,P-LIB,PP,PCTE,VOX,UPYD,PODEMOS-IU,convocatoria,participacion
0,28,1,La Acebeda,NaN,77,0,77,70,0,3,...,0,0,0,25,0,2,0,5,2019,90.909091
1,28,2,Ajalvir,NaN,3282,1,3283,2178,27,12,...,1,6,0,521,5,274,3,81,2019,66.341761
2,28,3,Alameda del Valle,NaN,169,0,169,140,0,0,...,0,0,0,41,0,8,0,3,2019,82.840237
3,28,4,El Álamo,NaN,6559,0,6559,4264,25,26,...,0,1,3,1102,2,537,5,236,2019,65.009910
4,28,5,Alcalá de Henares,NaN,136232,1,136233,89514,424,434,...,39,88,28,15953,85,8094,138,5306,2019,65.706547


#El mapa

Empezamos con un mapa coroplético de la participación en 2019.

Para asociar el dataframe y el JSON, hay que crear un campo id en el JSON que sea el código sin el último dígito

In [ ]:
with open(path + "Municipios.gjson", encoding='UTF-8') as geofile:
    j_file = json.load(geofile)

for feature in j_file['features']:
  feature['id'] = int(feature['properties']['GEOCODIGO'][:-1])

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(datos2019,
                           geojson=j_file,
                           mapbox_style="carto-positron",
                           center = {"lat": 40, "lon": -3},
                           color = 'participacion',
                           height = 800,
                           zoom= 6,
                           opacity=0.5,
                           locations = "Codmun",
                           hover_name = "Municipio"

                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.

Ahora cogemos los datos de 2015

In [ ]:
excel2015 =  pd.ExcelFile(path +'2015_Mesas.xls')

datos2015 = pd.read_excel(excel2015, 'Municipios')

datos2015 = datos2015[:-1].copy()
datos2015.Codmun = datos2015.Codmun.astype(np.int16)

datos2015['convocatoria'] = 2015
datos2015['participacion'] = 100 * datos2015['Votos_totales'] / datos2015['Censo total']

datos2015.head()

,Codcir,Codmun,Municipio,Unnamed: 3,Censo,Certif_alta,Censo total,Votos_totales,Votos_nulos,Votos_blancos,...,IUCM - LV,SAIn,PH,P.C.P.E.,FE de las JONS,PACMA,RECORTES CERO,VOX,convocatoria,participacion
0,28,2,Ajalvir,NaN,2914,0,2914,1958,28,28,...,56,0,3,2,4,23,1,10,2015,67.192862
1,28,3,Alameda del Valle,NaN,185,0,185,155,5,5,...,3,0,0,0,0,0,0,1,2015,83.783784
2,28,5,Alcalá de Henares,NaN,134182,2,134184,90495,901,1018,...,4009,82,118,97,127,875,107,692,2015,67.440977
3,28,6,Alcobendas,NaN,78273,1,78274,53115,479,565,...,1705,15,87,47,106,505,46,754,2015,67.857782
4,28,7,Alcorcón,NaN,124152,9,124161,87395,909,952,...,3467,36,100,104,106,934,147,727,2015,70.388447


Los unimos con los de 2019

In [ ]:
total = datos2019[['Municipio', 'participacion', 'convocatoria', 'Codmun']]
total = pd.concat((total, datos2015[['Municipio', 'participacion', 'convocatoria','Codmun']]))
total.head()

,Municipio,participacion,convocatoria,Codmun
0,La Acebeda,90.909091,2019,1
1,Ajalvir,66.341761,2019,2
2,Alameda del Valle,82.840237,2019,3
3,El Álamo,65.009910,2019,4
4,Alcalá de Henares,65.706547,2019,5


Mostrar el mapa

Quitamos las filas de residentes ausentes de Madrid

In [ ]:
total.participacion.min()

4.217903712167776

In [ ]:
# Encuentra la fila del minimo
municipio = total.loc[total['participacion'].idxmin()]

print(f"El municipio con la menor participación es:")
print(f"{municipio}")

El municipio con la menor participación es:
                      Municipio  participacion  convocatoria  Codmun
179  Residentes Ausentes Madrid       5.323904          2019     990
179  Residentes Ausentes Madrid       4.217904          2015     990


In [ ]:
total = total[total.Municipio!='Residentes Ausentes Madrid']

In [ ]:
total.sort_values(by="convocatoria", inplace = True)

fig = px.choropleth_mapbox(total,
                           geojson=j_file,
                           mapbox_style="carto-positron",
                           center = {"lat": 40, "lon": -3},
                           color = 'participacion',
                           height = 500,
                           zoom= 7,
                           opacity=0.5,
                           locations = 'Codmun',
                           range_color = [total.participacion.min(),total.participacion.max()],
                           animation_frame = 'convocatoria'
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Output hidden; open in https://colab.research.google.com to view.